In [1]:
'''
Algorithm: Follow the regularized leader - proximal

LB score : 0.668

Reference:
        http://www.eecs.tufts.edu/~dsculley/papers/ad-click-prediction.pdf

This code is based on tinrtgu famous beat the benchmark with less than 1MB of memory.
In short,
this is an adaptive-learning-rate sparse logistic-regression with
efficient L1-L2-regularization

https://www.kaggle.com/c/criteo-display-ad-challenge/discussion/10322

And adapted for the competition from
https://github.com/swapniel99/criteo

How to run in your terminal?

First save the file, example: FTRL_microsoft.py

With python3:
	python3 FTRL_microsoft.py

	To run with python3 you may need to install Python wrapper for MurmurHash (MurmurHash3) with:

	sudo pip install mmh3

With pypy:
	You can run the code with pypy as it will give you a huge speed-up (0nly took 5 minuties). To use the script
	with pypy on Ubuntu, first type

	sudo apt-get install pypy

	Then you need to get pure python MurmurHash3 implementation to run with pypy from the
	follwoing link and then put the file to current directory

	https://raw.githubusercontent.com/wc-duck/pymmh3/master/pymmh3.py

Then use the pypy interpreter to run the script

pypy FTRL_microsoft.py
'''

from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt


# try to run pure python MurmurHash3 implementation with pypy
# download pure pymmh3 from https://raw.githubusercontent.com/wc-duck/pymmh3/master/pymmh3.py

#from pymmh3 import hash # uncomment this line if you run this code with pypy, and comment the next import

# ... otherwise try with a fast c-implementation to run with python3...
from mmh3 import hash # comment this line if you run this code with pypy, and uncomment the previous import


# dataset parameters #################################################################
train = 'train.csv'  # path to training file
test = 'test.csv'  # path to testing file
submission = 'submission_ftrl.csv'  # path of to be outputted submission file


epoch = 1         # learn training data for N passes
logbatch = 10000  # batch log info after every N rows

# feature related parameters
signed = True  # Use signed hash? Set to False for to reduce number of hash calls
interaction = False  # whether to enable poly2 feature interactions
D = 2 ** 24    # number of weights use for learning

# model related parameters
lambda1 = 0.001  # L1 regularization, larger value means more regularized
lambda2 = 0.001  # L2 regularization, larger value means more regularized

if interaction:
    alpha = .04  # learning rate for sgd optimization
else:
    alpha = .1  # learning rate for sgd optimization
adapt = 1.  # Use adagrad, sets it as power of adaptive factor. >1 will amplify adaptive measure and vice versa
fudge = .4997  # Fudge factor (ratio of positive class)

# function definitions #######################################################

# A. Bounded logloss
# INPUT:
#     p: our prediction
#     y: real answer
# OUTPUT
#     logarithmic loss of p given y
def logloss(p, y):
    p = max(min(p, 1. - 10e-17), 10e-17)  # The bounds
    return -log(p) if y == 1. else -log(1. - p)


# B. Apply hash trick of the original csv row
# for simplicity, we treat both integer and categorical features as categorical
# INPUT:
#     csv_row: a csv dictionary, ex: {'Lable': '1', 'I1': '357', 'I2': '', ...}
#     D: the max index that we can hash to
# OUTPUT:
#     x: a list of indices that its value is 1
def get_x(csv_row, D):
    fullind = []
    for key, value in csv_row.items():
        s = key + '=' + value.lower()
        fullind.append(hash(s) % D)  # one-hot encode everything with hash trick

    if interaction == True:
        indlist2 = []
        for i in range(len(fullind)):
            for j in range(i + 1, len(fullind)):
                indlist2.append(fullind[i] ^ fullind[j])  # Creating interactions using XOR
        fullind = fullind + indlist2

    x = {}
    x[0] = 1  # 0 is the index of the bias term
    for index in fullind:
        if (index not in x):
            x[index] = 0
        if signed:
            x[index] += (1 if (hash(str(index)) % 2) == 1 else -1)  # Disable for speed
        else:
            x[index] += 1

    return x  # x contains indices of features that have a value as number of occurences


# C. Get probability estimation on x
# INPUT:
#     x: features
#     w: weights
# OUTPUT:
#     probability of p(y = 1 | x; w)
def get_p(x, w):
    wTx = 0.
    for i, xi in x.items():
        wTx += w[i] * xi  # w[i] * x[i]
    return 1. / (1. + exp(-max(min(wTx, 50.), -50.)))  # bounded sigmoid


# D. Update given model
# INPUT:
#     w: weights
#     n: a counter that counts the number of times we encounter a feature
#        this is used for adaptive learning rate
#     x: feature
#     p: prediction of our model
#     y: answer
# OUTPUT:
#     w: updated model
#     n: updated count
def update_w(w, g, x, p, y):
    for i, xi in x.items():
        # alpha / (sqrt(g) + 1) is the adaptive learning rate heuristic
        # (p - y) * x[i] is the current gradient
        # note that in our case, if i in x then x[i] = 1
        delreg = (lambda1 * ((-1.) if w[i] < 0. else 1.) + lambda2 * w[i]) if i != 0 else 0.
        delta = (p - y) * xi + delreg
        if adapt > 0:
            g[i] += delta ** 2
        w[i] -= delta * alpha / (sqrt(g[i]) ** adapt)  # Minimising log loss
    return w, g


# training and testing #######################################################

# initialize our model
w = [0.] * D  # weights
g = [fudge] * D  # sum of historical gradients

# start training a logistic regression model using on pass sgd
for e in range(epoch):
    loss = 0.
    lossb = 0.
    for t, row in enumerate(DictReader(open(train))):
        y = 1. if row['HasDetections'] == '1' else 0.

        del row['HasDetections']  # can't let the model peek the answer
        del row['MachineIdentifier']  # can't let the model peek the answer
        del row['Census_FirmwareVersionIdentifier'] 
        del row['Census_OEMModelIdentifier'] 
        del row['CityIdentifier']

        # main training procedure
        # step 1, get the hashed features
        x = get_x(row, D)
        # step 2, get prediction
        p = get_p(x, w)

        # for progress validation, useless for learning our model
        lossx = logloss(p, y)
        loss += lossx
        lossb += lossx
        if t % logbatch == 0 and t > 1:
            print('%s\tepoch-> %d\tencountered: %d\tcurrent whole logloss: %f\tcurrent batch logloss: %f' % (
            datetime.now(), e+1, t, loss / t, lossb / logbatch))
            lossb = 0.

        # step 3, update model with answer
        w, g = update_w(w, g, x, p, y)

# testing (build kaggle's submission file)
with open(submission, 'w') as outfile:
    outfile.write('MachineIdentifier,HasDetections\n')
    for t, row in enumerate(DictReader(open(test))):
        MachineIdentifier = row['MachineIdentifier']
        del row['MachineIdentifier']
        del row['Census_FirmwareVersionIdentifier'] 
        del row['Census_OEMModelIdentifier'] 
        del row['CityIdentifier']
        x = get_x(row, D)
        p = get_p(x, w)
        outfile.write('%s,%s\n' % (MachineIdentifier, str(p)))
        if t % logbatch == 0 and t > 1:
            print('%s\ttest rows encountered: %d' % (
                datetime.now(), t))

2018-12-27 07:10:12.954263	epoch-> 1	encountered: 10000	current whole logloss: 0.673937	current batch logloss: 0.673937
2018-12-27 07:10:30.091747	epoch-> 1	encountered: 20000	current whole logloss: 0.663931	current batch logloss: 0.653926
2018-12-27 07:10:47.707501	epoch-> 1	encountered: 30000	current whole logloss: 0.658527	current batch logloss: 0.647719
2018-12-27 07:11:04.618589	epoch-> 1	encountered: 40000	current whole logloss: 0.655233	current batch logloss: 0.645349
2018-12-27 07:11:16.650570	epoch-> 1	encountered: 50000	current whole logloss: 0.651713	current batch logloss: 0.637637
2018-12-27 07:11:30.295062	epoch-> 1	encountered: 60000	current whole logloss: 0.648538	current batch logloss: 0.632662
2018-12-27 07:11:42.014168	epoch-> 1	encountered: 70000	current whole logloss: 0.646748	current batch logloss: 0.636006
2018-12-27 07:11:53.809077	epoch-> 1	encountered: 80000	current whole logloss: 0.645021	current batch logloss: 0.632933
2018-12-27 07:12:03.056057	epoch-> 1	enc

2018-12-27 07:24:28.548660	epoch-> 1	encountered: 690000	current whole logloss: 0.628637	current batch logloss: 0.624038
2018-12-27 07:24:40.220985	epoch-> 1	encountered: 700000	current whole logloss: 0.628663	current batch logloss: 0.630450
2018-12-27 07:24:52.355897	epoch-> 1	encountered: 710000	current whole logloss: 0.628630	current batch logloss: 0.626347
2018-12-27 07:25:04.259991	epoch-> 1	encountered: 720000	current whole logloss: 0.628551	current batch logloss: 0.622908
2018-12-27 07:25:16.888778	epoch-> 1	encountered: 730000	current whole logloss: 0.628379	current batch logloss: 0.616032
2018-12-27 07:25:27.754658	epoch-> 1	encountered: 740000	current whole logloss: 0.628350	current batch logloss: 0.626179
2018-12-27 07:25:37.849104	epoch-> 1	encountered: 750000	current whole logloss: 0.628290	current batch logloss: 0.623853
2018-12-27 07:25:45.369971	epoch-> 1	encountered: 760000	current whole logloss: 0.628214	current batch logloss: 0.622539
2018-12-27 07:25:54.005762	epoch

2018-12-27 07:36:58.016163	epoch-> 1	encountered: 1370000	current whole logloss: 0.625933	current batch logloss: 0.618222
2018-12-27 07:37:10.627512	epoch-> 1	encountered: 1380000	current whole logloss: 0.625932	current batch logloss: 0.625843
2018-12-27 07:37:22.965708	epoch-> 1	encountered: 1390000	current whole logloss: 0.625903	current batch logloss: 0.621886
2018-12-27 07:37:35.189595	epoch-> 1	encountered: 1400000	current whole logloss: 0.625900	current batch logloss: 0.625540
2018-12-27 07:37:48.144740	epoch-> 1	encountered: 1410000	current whole logloss: 0.625825	current batch logloss: 0.615317
2018-12-27 07:38:00.190303	epoch-> 1	encountered: 1420000	current whole logloss: 0.625768	current batch logloss: 0.617704
2018-12-27 07:38:12.850479	epoch-> 1	encountered: 1430000	current whole logloss: 0.625732	current batch logloss: 0.620520
2018-12-27 07:38:25.050688	epoch-> 1	encountered: 1440000	current whole logloss: 0.625733	current batch logloss: 0.625882
2018-12-27 07:38:38.2047

2018-12-27 07:50:44.253478	epoch-> 1	encountered: 2050000	current whole logloss: 0.624551	current batch logloss: 0.615206
2018-12-27 07:50:56.342478	epoch-> 1	encountered: 2060000	current whole logloss: 0.624542	current batch logloss: 0.622739
2018-12-27 07:51:07.628461	epoch-> 1	encountered: 2070000	current whole logloss: 0.624503	current batch logloss: 0.616346
2018-12-27 07:51:18.454046	epoch-> 1	encountered: 2080000	current whole logloss: 0.624487	current batch logloss: 0.621158
2018-12-27 07:51:29.616408	epoch-> 1	encountered: 2090000	current whole logloss: 0.624465	current batch logloss: 0.620011
2018-12-27 07:51:40.245408	epoch-> 1	encountered: 2100000	current whole logloss: 0.624440	current batch logloss: 0.619271
2018-12-27 07:51:52.253117	epoch-> 1	encountered: 2110000	current whole logloss: 0.624434	current batch logloss: 0.623110
2018-12-27 07:52:03.436677	epoch-> 1	encountered: 2120000	current whole logloss: 0.624440	current batch logloss: 0.625596
2018-12-27 07:52:13.6500

2018-12-27 08:03:55.525847	epoch-> 1	encountered: 2730000	current whole logloss: 0.623741	current batch logloss: 0.624813
2018-12-27 08:04:07.332787	epoch-> 1	encountered: 2740000	current whole logloss: 0.623736	current batch logloss: 0.622306
2018-12-27 08:04:19.074780	epoch-> 1	encountered: 2750000	current whole logloss: 0.623728	current batch logloss: 0.621685
2018-12-27 08:04:31.633737	epoch-> 1	encountered: 2760000	current whole logloss: 0.623741	current batch logloss: 0.627148
2018-12-27 08:04:44.118456	epoch-> 1	encountered: 2770000	current whole logloss: 0.623726	current batch logloss: 0.619707
2018-12-27 08:04:56.292967	epoch-> 1	encountered: 2780000	current whole logloss: 0.623715	current batch logloss: 0.620617
2018-12-27 08:05:08.536541	epoch-> 1	encountered: 2790000	current whole logloss: 0.623698	current batch logloss: 0.619057
2018-12-27 08:05:21.602875	epoch-> 1	encountered: 2800000	current whole logloss: 0.623665	current batch logloss: 0.614239
2018-12-27 08:05:33.7875

...

2018-12-27 09:46:19.926137	test rows encountered: 1280000
2018-12-27 09:46:26.321685	test rows encountered: 1290000
2018-12-27 09:46:32.888635	test rows encountered: 1300000
2018-12-27 09:46:39.070129	test rows encountered: 1310000
2018-12-27 09:46:45.261026	test rows encountered: 1320000
2018-12-27 09:46:51.551600	test rows encountered: 1330000
2018-12-27 09:46:57.540762	test rows encountered: 1340000
2018-12-27 09:47:03.721683	test rows encountered: 1350000
2018-12-27 09:47:09.895301	test rows encountered: 1360000
2018-12-27 09:47:16.497303	test rows encountered: 1370000
2018-12-27 09:47:22.960443	test rows encountered: 1380000
2018-12-27 09:47:29.135366	test rows encountered: 1390000
2018-12-27 09:47:35.557161	test rows encountered: 1400000
2018-12-27 09:47:41.722572	test rows encountered: 1410000
2018-12-27 09:47:47.893706	test rows encountered: 1420000
2018-12-27 09:47:53.993564	test rows encountered: 1430000
2018-12-27 09:48:00.644406	test rows encountered: 1440000
2018-12-27 09:

2018-12-27 09:56:49.465561	test rows encountered: 2700000
2018-12-27 09:56:55.583316	test rows encountered: 2710000
2018-12-27 09:57:01.498975	test rows encountered: 2720000
2018-12-27 09:57:08.044929	test rows encountered: 2730000
2018-12-27 09:57:14.055760	test rows encountered: 2740000
2018-12-27 09:57:20.134836	test rows encountered: 2750000
2018-12-27 09:57:26.199845	test rows encountered: 2760000
2018-12-27 09:57:32.445977	test rows encountered: 2770000
2018-12-27 09:57:38.441360	test rows encountered: 2780000
2018-12-27 09:57:44.878911	test rows encountered: 2790000
2018-12-27 09:57:50.873994	test rows encountered: 2800000
2018-12-27 09:57:57.028787	test rows encountered: 2810000
2018-12-27 09:58:02.432937	test rows encountered: 2820000
2018-12-27 09:58:08.377572	test rows encountered: 2830000
2018-12-27 09:58:14.148944	test rows encountered: 2840000
2018-12-27 09:58:19.067107	test rows encountered: 2850000
2018-12-27 09:58:23.553497	test rows encountered: 2860000
2018-12-27 09:

2018-12-27 10:11:12.161378	test rows encountered: 4120000
2018-12-27 10:11:18.250676	test rows encountered: 4130000
2018-12-27 10:11:24.340844	test rows encountered: 4140000
2018-12-27 10:11:30.130992	test rows encountered: 4150000
2018-12-27 10:11:35.349566	test rows encountered: 4160000
2018-12-27 10:11:39.427157	test rows encountered: 4170000
2018-12-27 10:11:45.224241	test rows encountered: 4180000
2018-12-27 10:11:51.866608	test rows encountered: 4190000
2018-12-27 10:11:58.664974	test rows encountered: 4200000
2018-12-27 10:12:05.089004	test rows encountered: 4210000
2018-12-27 10:12:10.855144	test rows encountered: 4220000
2018-12-27 10:12:16.999741	test rows encountered: 4230000
2018-12-27 10:12:23.240009	test rows encountered: 4240000
2018-12-27 10:12:29.375090	test rows encountered: 4250000
2018-12-27 10:12:34.307145	test rows encountered: 4260000
2018-12-27 10:12:37.500257	test rows encountered: 4270000
2018-12-27 10:12:40.978944	test rows encountered: 4280000
2018-12-27 10:

2018-12-27 10:24:37.566989	test rows encountered: 5540000
2018-12-27 10:24:43.246822	test rows encountered: 5550000
2018-12-27 10:24:49.276765	test rows encountered: 5560000
2018-12-27 10:24:55.467046	test rows encountered: 5570000
2018-12-27 10:25:01.472444	test rows encountered: 5580000
2018-12-27 10:25:07.656488	test rows encountered: 5590000
2018-12-27 10:25:14.045677	test rows encountered: 5600000
2018-12-27 10:25:20.191261	test rows encountered: 5610000
2018-12-27 10:25:26.438168	test rows encountered: 5620000
2018-12-27 10:25:33.096983	test rows encountered: 5630000
2018-12-27 10:25:39.580774	test rows encountered: 5640000
2018-12-27 10:25:44.793193	test rows encountered: 5650000
2018-12-27 10:25:50.860963	test rows encountered: 5660000
2018-12-27 10:25:57.314617	test rows encountered: 5670000
2018-12-27 10:26:04.231546	test rows encountered: 5680000
2018-12-27 10:26:10.488365	test rows encountered: 5690000
2018-12-27 10:26:16.743859	test rows encountered: 5700000
2018-12-27 10:

2018-12-27 10:38:06.475658	test rows encountered: 6960000
2018-12-27 10:38:10.529409	test rows encountered: 6970000
2018-12-27 10:38:14.865690	test rows encountered: 6980000
2018-12-27 10:38:18.905070	test rows encountered: 6990000
2018-12-27 10:38:22.508233	test rows encountered: 7000000
2018-12-27 10:38:26.133165	test rows encountered: 7010000
2018-12-27 10:38:29.510738	test rows encountered: 7020000
2018-12-27 10:38:33.509896	test rows encountered: 7030000
2018-12-27 10:38:37.004267	test rows encountered: 7040000
2018-12-27 10:38:40.603720	test rows encountered: 7050000
2018-12-27 10:38:44.361028	test rows encountered: 7060000
2018-12-27 10:38:48.313731	test rows encountered: 7070000
2018-12-27 10:38:52.133251	test rows encountered: 7080000
2018-12-27 10:38:56.152482	test rows encountered: 7090000
2018-12-27 10:38:59.557053	test rows encountered: 7100000
2018-12-27 10:39:02.643381	test rows encountered: 7110000
2018-12-27 10:39:05.742865	test rows encountered: 7120000
2018-12-27 10: